# Marlin Basics

Marlin is a high-performance, fully tensorized simulation tool built on top of the MOOSE (Multiphysics Object-Oriented Simulation Environment) framework. As a result, the input file syntax is very similar to that of MOOSE. It leverages LibTorch (the C++ frontend of PyTorch) to perform tensor operations on heterogeneous hardware (CPUs and GPUs).

Marlin is designed as a command-line tool where simulations are defined using structured input files. These files control every aspect of the simulation, including:
- **Domain and Geometry**: Defining the simulation grid and physical dimensions.
- **Tensor Buffers**: Allocating memory for field variables.
- **Compute Objects**: Defining the physics and evolution equations.
- **Boundary Conditions**: Specifying behavior at domain boundaries.
- **Execution Control**: Managing time-stepping and output generation.

While simple geometries can be defined directly in the input file, we recommend using Python scripting for complex setups and ParaView for high-fidelity 3D visualization.

In this tutorial, we will guide you through the basic syntax of a Marlin input file, starting with the domain definition.

### The `[Domain]` Block

The `[Domain]` block is the starting point of any Marlin simulation. It defines the spatial discretization (mesh) and the hardware resources to be used.

**Key Parameters:**
- `dim`: The dimension of the simulation (1, 2, or 3).
- `nx`, `ny`, `nz`: The number of grid points in the x, y, and z directions.
- `xmax`, `ymax`, `zmax`: The physical size of the domain.
- `device_names`: The hardware device(s) to run on (e.g., `'cpu'`, `'cuda'`, `'mps'`).
- `mesh_mode`: Specifies how the mesh is handled (e.g., `DUMMY` to avoid constructing a standard MOOSE mesh).

For a more detailed description of the `Domain` object, please refer to `doc/source/actions/DomainAction.md`.

Here is an example of a 2D domain configuration:

In [45]:
# Define the Domain block
domain_block = """
[Domain]
  dim = 2
  nx = 100
  ny = 100
  mesh_mode = DUMMY
[]
"""

print(domain_block)


[Domain]
  dim = 2
  nx = 100
  ny = 100
  mesh_mode = DUMMY
[]



### The `[Stencil]` Block

The `[Stencil]` block defines the discrete velocity set used in the LBM simulation.

**What is a Stencil?**
In LBM, the continuous velocity space is discretized into a finite set of directions and speeds. The stencil (also known as the lattice or descriptor in the literature) defines these discrete velocities ($e_i$), their associated weights ($w_i$), the transformation matrix $\mathbf{M}$, its inverse, the diagonal relaxation matrix, and other properties.

**Why is it important?**
The choice of stencil is crucial as it determines the accuracy, stability, and computational cost of the simulation. It provides the link between the mesoscopic particle distribution functions and the macroscopic fluid properties (density, velocity, temperature, order parameter).

**Available Stencils in Marlin:**
- `LBMD2Q9`: 2 Dimensions, 9 Velocities. The standard stencil for 2D incompressible flow.
- `LBMD3Q19`: 3 Dimensions, 19 Velocities. A common choice for 3D flows, offering a good balance between accuracy and efficiency.
- `LBMD3Q27`: 3 Dimensions, 27 Velocities. More accurate but computationally more expensive; a recommended choice for high Reynolds number flows.

**Configuration:**
You define a sub-block with a user-defined name (e.g., `d2q9`) and set the `type` parameter to one of the available stencils.

In [46]:
# Define the Stencil block
stencil_block = """
[Stencil]
  [d2q9]
    type = LBMD2Q9
  []
[]
"""

print(stencil_block)


[Stencil]
  [d2q9]
    type = LBMD2Q9
  []
[]



### The `[TensorBuffers]` Block

The `[TensorBuffers]` block is responsible for defining and allocating memory for the simulation variables (tensors). Internally, these are `torch::Tensor` objects with dimensions determined by the `[Domain]` and `[Stencil]` settings. Memory is automatically allocated on the hardware device(s) specified in the `[Domain]` block.

**Key Parameters:**
- `type`: Typically set to `LBMTensorBuffer` for LBM simulations.
- `buffer_type`: Defines the shape and role of the tensor.
    - `df`: **Distribution Function**. Shape `[ny, nx, nz, Q]`. Stores particle distribution functions ($f$, $f^{eq}$, etc.).
    - `mv`: **Macroscopic Vector**. Shape `[ny, nx, nz, dim]`. Stores vector fields such as velocity, force, etc.
    - `ms`: **Macroscopic Scalar**. Shape `[ny, nx, nz]`. Stores scalar fields such as density, temperature, speed, etc.

**Note:** In 2D simulations, Marlin automatically sets `nz=1`. This design unifies 2D and 3D implementations in the backend, requiring no extra configuration from the user.

In this example, we define buffers for:
- `f`: The particle distribution function.
- `feq`: The equilibrium distribution function.
- `fpc`: The post-collision distribution function.
- `velocity`: The macroscopic velocity vector.
- `density`: The macroscopic density scalar.
- `speed`: The magnitude of velocity.

In [47]:
tensor_buffers_block = """
[TensorBuffers]
  [f]
    type = LBMTensorBuffer
    buffer_type = df
  []
  [feq]
    type = LBMTensorBuffer
    buffer_type = df
  []
  [fpc]
    type = LBMTensorBuffer
    buffer_type = df
  []
  [velocity]
    type=LBMTensorBuffer
    buffer_type = mv
  []
  [density]
    type=LBMTensorBuffer
    buffer_type = ms
  []
  [speed]
    type=LBMTensorBuffer
    buffer_type = ms
  []
[]
"""

print(tensor_buffers_block)


[TensorBuffers]
  [f]
    type = LBMTensorBuffer
    buffer_type = df
  []
  [feq]
    type = LBMTensorBuffer
    buffer_type = df
  []
  [fpc]
    type = LBMTensorBuffer
    buffer_type = df
  []
  [velocity]
    type=LBMTensorBuffer
    buffer_type = mv
  []
  [density]
    type=LBMTensorBuffer
    buffer_type = ms
  []
  [speed]
    type=LBMTensorBuffer
    buffer_type = ms
  []
[]



### Loading External Data

In addition to allocating empty buffers, `[TensorBuffers]` can also initialize tensors from external files. This is particularly useful for defining complex geometries, such as porous media, where the geometry is represented by a binary mask (0 for fluid, 1 for solid).

**Key Parameters for Loading Data:**
- `file`: Path to an HDF5 file containing the tensor data. The file should contain a dataset with the same name as the buffer.
- `is_integer`: Set to `true` if the tensor stores integer values (e.g., material IDs, region masks). Defaults to `false`.

This feature allows you to generate complex geometries using external tools (like Python or MATLAB) and load them directly into Marlin.

In [48]:
binary_media_example = """
[TensorBuffers]
  [binary_media]
    type = LBMTensorBuffer
    file = binary_media.h5
    buffer_type = ms
    is_integer = true
  []
[]
"""
print(binary_media_example)


[TensorBuffers]
  [binary_media]
    type = LBMTensorBuffer
    file = binary_media.h5
    buffer_type = ms
    is_integer = true
  []
[]



### The `[TensorComputes]` Block

The `[TensorComputes]` block is the computational heart of the simulation. It defines the sequence of compute objects that evolve the system state. This block is organized into three primary sub-blocks, each serving a distinct phase of the simulation lifecycle.

#### 1. The `[Initialize]` Sub-block
This block runs **once** at the very beginning of the simulation. Its purpose is to set the initial conditions for the tensor buffers.
- **Role**: Initialize macroscopic variables (density, velocity) and the corresponding distribution functions.
- **Common Objects**:
    - `LBMConstantTensor`: Sets a buffer to a constant value (scalar or vector).
    - `LBMEquilibrium`: Computes the initial equilibrium distribution ($f^{eq}$) from the initial density and velocity.

In [49]:
tensor_computes_initial_block = """
[TensorComputes/Initialize]
  [initial_density]
    type = LBMConstantTensor
    buffer = density
    constants = 1.0
  []
  [initial_velocity]
    type = LBMConstantTensor
    buffer = velocity
    constants = '0.0 0.0'
  []
  [initial_equilibrium]
    type = LBMEquilibrium
    buffer = feq
    bulk = density
    velocity = velocity
  []
  [initial_distribution]
    type = LBMEquilibrium
    buffer = f
    bulk = density
    velocity = velocity
  []
  [initial_distribution_pc]
    type = LBMEquilibrium
    buffer = fpc
    bulk = density
    velocity = velocity
  []
[]
"""
print(tensor_computes_initial_block)


[TensorComputes/Initialize]
  [initial_density]
    type = LBMConstantTensor
    buffer = density
    constants = 1.0
  []
  [initial_velocity]
    type = LBMConstantTensor
    buffer = velocity
    constants = '0.0 0.0'
  []
  [initial_equilibrium]
    type = LBMEquilibrium
    buffer = feq
    bulk = density
    velocity = velocity
  []
  [initial_distribution]
    type = LBMEquilibrium
    buffer = f
    bulk = density
    velocity = velocity
  []
  [initial_distribution_pc]
    type = LBMEquilibrium
    buffer = fpc
    bulk = density
    velocity = velocity
  []
[]



#### 2. The `[Solve]` Sub-block
This block runs **at every iteration**. It defines the core physics loop of the Lattice Boltzmann Method. The order of objects here do not matter since Marlin carries out internal dependency resolution. We will go over the available compute objects in more details in the upcoming tutorials.

In [50]:
tensor_computes_solve_block = """
[TensorComputes/Solve]
  [equilibrium]
    type=LBMEquilibrium
    buffer = feq
    bulk = density
    velocity = velocity
  []
  [collision]
    type=LBMBGKCollision
    buffer = fpc
    f = f
    feq = feq
    tau0 = 1.0
  []
  [density]
    type = LBMComputeDensity
    buffer = density
    f = f
  []
  [velocity]
    type = LBMComputeVelocity
    buffer = velocity
    f = f
    rho = density
    add_body_force = true
    body_force_x = 0.0001
  []
  [speed]
    type = LBMComputeVelocityMagnitude
    buffer = speed
    velocity = velocity
  []
  [residual]
    type = LBMComputeResidual
    buffer = speed
    speed = speed
  []
[]
"""

print(tensor_computes_solve_block)


[TensorComputes/Solve]
  [equilibrium]
    type=LBMEquilibrium
    buffer = feq
    bulk = density
    velocity = velocity
  []
  [collision]
    type=LBMBGKCollision
    buffer = fpc
    f = f
    feq = feq
    tau0 = 1.0
  []
  [density]
    type = LBMComputeDensity
    buffer = density
    f = f
  []
  [velocity]
    type = LBMComputeVelocity
    buffer = velocity
    f = f
    rho = density
    add_body_force = true
    body_force_x = 0.0001
  []
  [speed]
    type = LBMComputeVelocityMagnitude
    buffer = speed
    velocity = velocity
  []
  [residual]
    type = LBMComputeResidual
    buffer = speed
    speed = speed
  []
[]



#### 3. The `[Boundary]` Sub-block
This block also runs **at every iteration**, typically after the streaming step (which is handled separately by the solver).
- **Role**: Enforce boundary conditions at the edges of the domain or on internal obstacles.
- **Common Objects**:
    - `LBMBounceBack`: Implements the no-slip condition (velocity = 0) for solid walls.
    - `LBMDirichletBC`: Sets a fixed value for a macroscopic variable (e.g., fixed density inlet).
    - `LBMNeumannBC`: Sets a fixed gradient (flux) for a variable.

In [51]:
tensor_computes_boundary_block = """
[TensorComputes/Boundary]
  [top]
    type = LBMBounceBack
    buffer = f
    f_old = fpc
    boundary = top
  []
  [bottom]
    type = LBMBounceBack
    buffer = f
    f_old = fpc
    boundary = bottom
  []
[]
"""

print(tensor_computes_boundary_block)


[TensorComputes/Boundary]
  [top]
    type = LBMBounceBack
    buffer = f
    f_old = fpc
    boundary = top
  []
  [bottom]
    type = LBMBounceBack
    buffer = f
    f_old = fpc
    boundary = bottom
  []
[]



### The `[TensorSolver]` Block

The `[TensorSolver]` block defines the streaming step of the Lattice Boltzmann Method. In Marlin, streaming is handled separately from the collision and force calculations.

**Key Parameters:**
- `type`: Must be `LBMStream`.
- `buffer`: The destination buffer (post-streaming distribution function, usually `f`).
- `f_old`: The source buffer (pre-streaming and post-collision distribution function, usually `fpc` from the collision step).

This block effectively moves particle populations to their neighboring lattice sites according to their discrete velocities.

In [52]:
# Define the TensorSolver block
tensor_solver_block = """
[TensorSolver]
  type = LBMStream
  buffer = f
  f_old = fpc
[]
"""

print(tensor_solver_block)


[TensorSolver]
  type = LBMStream
  buffer = f
  f_old = fpc
[]



### The `[Problem]` Block

The `[Problem]` block configures global simulation parameters and manages the overall problem execution.

**Key Parameters:**
- `type`: Must be `LatticeBoltzmannProblem` for LBM simulations.
- `substeps`: The number of LBM time steps to execute for each "MOOSE time step" (defined in `[Executioner]`). LBM typically requires many small time steps, so grouping them is generally recommended for output storage efficiency. The outputs will be executed after every LBM `substeps`.
- `scalar_constant_names` & `scalar_constant_values`: Define global constants (e.g., relaxation time, initial constants, reference constants) that can be used in other blocks.
- `print_debug_output`: Defaults to `false`. If set to `true`, it prints the execution order of compute objects in the `[TensorComputes/Solve]` block.
- `binary_media`: If the geometry is loaded as a tensor buffer, then this parameter should be equal to the name of that buffer. We will be going over external geometry in upcoming tutorials.

In [53]:
# Define the Problem block
problem_block = """
[Problem]
  type = LatticeBoltzmannProblem
  substeps = 1000
  scalar_constant_names = 'tau0 rho0'
  scalar_constant_values = '0.85 1.0'
  # binary_media = binary_media
[]
"""

print(problem_block)


[Problem]
  type = LatticeBoltzmannProblem
  substeps = 1000
  scalar_constant_names = 'tau0 rho0'
  scalar_constant_values = '0.85 1.0'
  # binary_media = binary_media
[]



### The `[Executioner]` Block

The `[Executioner]` block controls the overall time-stepping of the simulation.

**Key Parameters:**
- `type`: `Transient` for time-dependent simulations.
- `num_steps`: The number of "outer" time steps. The total number of LBM iterations is `num_steps` $\times$ `substeps`.

In [54]:
# Define the Executioner block
executioner_block = """
[Executioner]
  type = Transient
  num_steps = 2
[]
"""

print(executioner_block)


[Executioner]
  type = Transient
  num_steps = 2
[]



### The `[TensorOutputs]` Block

The `[TensorOutputs]` block defines how and when simulation results are written to disk.

**Key Parameters:**
- `type`: `XDMFTensorOutput` is recommended for large datasets as it writes heavy data to HDF5 and lightweight metadata to XDMF (readable by ParaView).
- `buffer`: A list of tensor buffers to output (e.g., `'density velocity'`).
- `output_mode`: Specifies whether data is cell-centered (`Cell`) or node-centered (`Node`). LBM data is typically cell-centered.
- `enable_hdf5`: Must be set to `true` for XDMF output.

In [55]:
# Define the TensorOutputs block
tensor_outputs_block = """
[TensorOutputs]
  [xdmf2]
    type = XDMFTensorOutput
    buffer = 'velocity'
    output_mode = 'Cell'
    enable_hdf5 = true
  []
[]
"""

print(tensor_outputs_block)


[TensorOutputs]
  [xdmf2]
    type = XDMFTensorOutput
    buffer = 'velocity'
    output_mode = 'Cell'
    enable_hdf5 = true
  []
[]



## Assembling and Running the Simulation

Now that we have defined all the components, we can assemble the full input file and run the simulation.

We will write the complete configuration to a file named `001_basics.i` and then execute it using `marlin-opt`.

It is best to execute the input file from the command line via `../../../marlin-opt -i 001_basics.i`, but it can also be done using Python.

In [57]:
import subprocess
import os

# Combine all blocks into a single string
full_input_file = domain_block + stencil_block + tensor_buffers_block + \
                  tensor_computes_initial_block + tensor_computes_solve_block + \
                  tensor_computes_boundary_block + tensor_solver_block + \
                  problem_block + executioner_block + tensor_outputs_block

# Write to file
with open('001_basics.i', 'w') as f:
    f.write(full_input_file)

print("Input file '001_basics.i' created successfully.")

# Path to the executable
marlin_exec = os.path.abspath('../../../marlin-opt')

# Check if executable exists (make sure to build Marlin before running)
if not os.path.exists(marlin_exec):
    print(f"Error: Executable not found at {marlin_exec}")
else:
    print(f"Running simulation with {marlin_exec}...")
    # Run the simulation
    process = subprocess.Popen([marlin_exec, '-i', '001_basics.i'], 
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE,
                               text=True)

    # Print output in real-time (or wait and print)
    stdout, stderr = process.communicate()

    if process.returncode == 0:
        print("Simulation completed successfully!")
        # Print the last few lines of output
        print("\n".join(stdout.splitlines()[-10:]))
    else:
        print("Simulation failed!")
        print(stderr)

Input file '001_basics.i' created successfully.
Running simulation with /home/nrustamo/projects/marlin/marlin-opt...
Simulation completed successfully!
Lattice Boltzmann Substep 993, Residual 0.000885188
Lattice Boltzmann Substep 994, Residual 0.000884223
Lattice Boltzmann Substep 995, Residual 0.00088326
Lattice Boltzmann Substep 996, Residual 0.0008823
Lattice Boltzmann Substep 997, Residual 0.000881341
Lattice Boltzmann Substep 998, Residual 0.000880383
Lattice Boltzmann Substep 999, Residual 0.000879428
 0 Nonlinear |R| = 0.000000e+00
 Solve Converged!
Finished Executing                                                                       [ 21.87 s] [  247 MB]
Simulation completed successfully!
Lattice Boltzmann Substep 993, Residual 0.000885188
Lattice Boltzmann Substep 994, Residual 0.000884223
Lattice Boltzmann Substep 995, Residual 0.00088326
Lattice Boltzmann Substep 996, Residual 0.0008823
Lattice Boltzmann Substep 997, Residual 0.000881341
Lattice Boltzmann Substep 998, Res